In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"US/Central\"")

In [ ]:
hex_project_id = _hex_json.loads("\"5171d1fe-7310-437f-a379-f3121a2be2c1\"")

In [ ]:
hex_project_name = _hex_json.loads("\"Tag - Voted\"")

In [ ]:
hex_status = _hex_json.loads("\"\"")

In [ ]:
hex_categories = _hex_json.loads("[]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

## Step 0: Instructions + Configuration

This template helps you add tags in Reach. Please take care to modify it as little as possible to promote standardization. **You should only need to alter steps 0 and 1.**

As an overview:

1. Create a tag within the Reach UI admin panel (could be done through API but seems unneccessary for one-time operation). Be sure to chose options:
  + Locked: Yes (so users can't remove)
  + Show in Quick Filters: No (to not clutter the interface)
2. Edit the following fields in the chunk directly below:
  + `tag_name`: The name exactly as created in the Reach admin panel.
  + `tag_id`: Enter if known (don't worry if not known - code can figure it out)
  + `tag_action`: 'added' to add tag, 'removed' to remove tag
3. Modify the `tag_targets` CTE (standalone code chunk) of the SQL query to pull population of interest
4. Run and check for successful execution in Step 5
5. Schedule for a daily run or at whatever cadence you wish to update tags 

In [ ]:
# Once we know the tag information, we can hardcode it here
tag_name = 'Has Voted' # ensure name is exactly as written in Reach admin panel
tag_id = 'QJPQ496M' # set to '' if unknown and code will determine
tag_action = ['added','removed'][0] # set index to 0 for 'added' to add tag or 1 for 'removed' to remove tag

## Step 1: Define Target Population

Modify *only* the targets and exclusion CTE in the below query for the populations of interest

In [ ]:
# import jinja2
# raw_query = """
#     select *
#     from reach_tags.tag_voters
#     
#     /*
#       reach_id as person_id,
#       'Reach ID' as person_id_type
#     from 
#       `two-million-texans.reach_phoenix.people_votes` as people_votes
#     where 
#       (date_ballot_received is not null) or
#       (date_early_voted is not null)
#       */
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     -- Note: this script uses Hex-specific logic for parameterized SQL
#     -- See details at: https://learn.hex.tech/docs/logic-cell-types/sql-cells/sql-parameterization
#     
#     with 
#     targets as (
#     
#         select * from tag_targets
#     
#     ),
#     has_tag as (
#       select
#         cast(person_id as int64) as person_id,
#         case when message = 'added' then 1 else null end as ind_has_tag
#       from reach_tags.tag_log
#       where 
#         tag_name = {{tag_name}} and
#         status = 'success'
#         qualify row_number() over (partition by person_id order by run_finished_ts desc) = 1
#     )
#     select 
#       targets.*,
#       {{tag_action}} as action
#     from 
#       targets
#       left join
#       has_tag
#       using (person_id)
#     where 
#       {% if tag_action == 'added' %}
#       ind_has_tag is null -- find people who don't have tag to add
#       {% else %}
#       ind_has_tag = 1 -- find people who have tag to remove
#       {% endif %}
#     
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
import pandas as pd
persons = df_totag.to_dict(orient = 'records')

## Step 2: Retrieve ephemeral API Token

No actions required

In [ ]:
import requests
import pandas as pd

# Modified from: https://github.com/ossoffforsenate/relational_toolkit/blob/main/reach/tag_uncontactable.py

# Authenticate with OAuth2
# API endpoint docs: https://api.reach.vote/#tag/OAuth2
creds_payload = {"username": reach_api_username, 
                 "password": reach_api_password}
creds_req = requests.post("https://api.reach.vote/oauth/token", creds_payload)
creds_res = creds_req.json()["access_token"]
headers = {"Authorization": f"Bearer {creds_res}"}

## Step 3: Retrieve Tag ID if Unknown

The following step makes a call to the API to discern the tag_id.

If `tag_id` is set above, the rest of the code will be skipped.

If `tag_id` is set to `''` (empty string), this code chunk will figure out the appropriate value

In [ ]:
# Get correct tag metadata
# If needed, the following chunk will find the appropriate tag_id
if tag_id == '':
  tags_req = requests.get("https://api.reach.vote/api/v1/tags", headers = headers)
  tags_res = tags_req.json()
  tag_id = [t for t in tags_res['tags'] if t['name'] == tag_name][0]['id']

## Step 4: Iterate over Persons to Add Tag

No changes needed

In [ ]:
# Number of groups needed
import math
per_bin = 3000
n_bins = math.ceil(len(persons) / per_bin)

# Create status code audit
status_code = []
results = []

# Prep endpoint
endpoint = f"https://api.reach.vote/api/v1/tags/{tag_id}"

# Iterate over rows 
for i in range(n_bins):
    
    print(f'Starting chunk {i+1} of {n_bins}')
    people_bin = persons[i*per_bin : (i+1)*per_bin]
    payload = {"people": people_bin}
    req = requests.put(endpoint, json = payload, headers = headers)
    status_code.append(req.status_code)
    res = req.json()
    results.append(res)

Starting chunk 1 of 2
Starting chunk 2 of 2


## Step 5: Audit execution

Confirm all submissions were successful.

If not, inspect the `results` object to debug.

In [ ]:
# Audit for success
print(f'Number of People: {len(persons)}')
print(f'Number of Batches: {n_bins}')
print(f'Succeeded: {sum([s == 200 for s in status_code])}')
print(f'All succeeded?: {all(s == 200 for s in status_code)}')

Number of People: 4995
Number of Batches: 2
Succeeded: 2
All succeeded?: True


## Step 6: Logging

In [ ]:
results_people = []

for r in results:
    res_list = [p for p in r['people']]
    results_people += res_list

df_totag_report = pd.DataFrame.from_dict(results_people)

In [ ]:
import datetime
import pytz

df_totag_report = df_totag_report[['person_id', 'status','message']]
df_totag_report['tag_name'] = tag_name
df_totag_report['run_finished_ts'] = datetime.datetime.now(pytz.timezone('US/Central'))

In [ ]:
df_totag_report.groupby('status')['person_id'].count()

status
success    4995
Name: person_id, dtype: int64

In [ ]:
# if hex_run_context in ["scheduled"]:
#     import hextoolkit
#     hex_data_connection = hextoolkit.get_data_connection("2MT BigQuery 2023")
#     writeback_metadata = hex_data_connection.write_dataframe(df=df_totag_report, database=None, schema="reach_tags", table="tag_log", overwrite=False)